In [ ]:
# RESTART KERNEL AFTER INSTALLING
!pip install PyPDF2

PULL INFO FOR ALL IRS WRITTEN DETERMINATIONS TO FORM OUR DATABASE

NOTE THAT SCRAPING CODE WAS GENERATED BY GPT API, EDITED BY ME

In [ ]:
#IMPORT NECESSARY PACKAGES
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os


In [ ]:
# DEFINE VARIABLES FOR GOOGLE CLOUD INTEGRATION 
REGION = 'us-central1' # NEEDS TO BE SAME REGION AS BUCKETS, OTHER NBs, INSTANCE, ETC.
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
BUCKET = 'gs://legal_precedents_test' # REPLACE WITH YOUR OWN BUCKET NAME

In [ ]:
# DEFINE FUNCTION TO SCRAPE THE TABLE ON THE IRS WEBPAGE, GET CORE INFO ON CASES
def scrape_table(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'class': 'tablesaw tablesaw-stack table table-hover table-striped pup-table'})
    rows = table.find_all('tr')
    
    data = []
    for row in rows[1:]:
        cells = row.find_all('td')
        number = cells[0].find('a').text.strip()
        uilc = cells[1].text.strip()
        subject = cells[2].text.strip()
        release_date = cells[3].text.strip()
        pdf_url = cells[0].find('a')['href']
        data.append([number, uilc, subject, release_date, pdf_url])
    
    return data, soup

In [ ]:
# SCRAPE THE FIRST PAGE
url = 'https://www.irs.gov/written-determinations'
first_time = scrape_table(url)
data = first_time[0]
soup = first_time[1]

# AND SUBSEQUENT PAGES IF AVAILABLE 
next_page = soup.find('li', {'class': 'pager__item pager__item--next'})

while next_page:
    next_url = url + next_page.find('a').get('href','') if next_page.find('a') else ''
    instance = scrape_table(next_url)
    new_data = instance[0]
    new_soup = instance[1]
    data += new_data
    next_page = new_soup.find('li', {'class': 'pager__item pager__item--next'})

# CONVERT TO PANDAS DATAFRAME
df = pd.DataFrame(data, columns=['Number', 'UILC', 'Subject', 'Release Date', 'pdf'])


In [ ]:
#CHECK DF
df

In [ ]:
# AND SAVE LOCAL CSV
df.to_csv("scraped_determinations.csv")

In [ ]:
# CREATE BUCKET IF YOU HAVEN'T ALREADY
!gsutil mb -l {REGION} {BUCKET} 

In [ ]:
# SAVE CSV IN CLOUD STORAGE TO MAKE SURE YOU HAVE IT MOVING FORWARD
!gsutil cp scraped_determinations.csv {BUCKET}/scraped_determinations.csv

SAVE PDF OF ALL WDs FOR VECTOR SEARCH IN LATER NOTEBOOK

In [ ]:
from PyPDF2 import PdfReader
import os
import requests

In [ ]:
# NOT ALL PDFs PULLED ARE USEABLE - WANT TO TOSS THE CORRUPTED ONES
def checkFile(fullfile):
    with open(fullfile, 'rb') as f:
        try:
            pdf = PdfReader(f)
            info = pdf.metadata
            if info:
                return True
            else:
                return False
        except:
            return False

In [ ]:
# # IF YOU NEED TO RESTART KERNEL READ-IN LOCAL FILE INSTEAD OF RE-RUNNING SCRAPE
# df = pd.read_csv("scraped_determinations.csv")

In [ ]:
# NOTE THAT WE ARE WORKING WITH A NEW BUCKET
BUCKET = 'gs://irs_written_determinations_test/' # CHANGE TO YOUR OWN NAMING CONVENTION

In [ ]:
# CREATE BUCKET FOR PDF STORAGE
!gsutil mb -l {REGION} {BUCKET} 

In [ ]:
# USE URL TO GET LOCAL COPY OF PDF OF WRITINGS ON THE ACTUAL CASE
# NOTE THAT INITIAL SCRAPE PULLED THE NAME OF THE PDF THAT THE WEBSITE LINKS TO FOR THOSE WANTING TO SEE WRITING
counter = 1
for pdf in list(set(df['pdf'].values.tolist())):
    print(counter) #YOU CAN DELETE THIS PART - I PERSONALLY LIKE IT TO KEEP TRACK OF PROGRESS
    url = "https://www.irs.gov" + pdf
    
    local_file_path = pdf[12:]

    response = requests.get(url)

    with open(local_file_path, "wb") as file:
        file.write(response.content)
    
    # ONLY ADDING USEABLE FILES TO OUR BUCKET FOR SEMANTIC SEARCH LATER
    if checkFile(local_file_path):
        os.system(f"gsutil cp {local_file_path} {BUCKET}")
        os.system(f"rm {local_file_path}")
    counter +=1

In [ ]:
# CHECK HOW MANY FILES ARE IN THE BUCKET; 
# I'VE TYPICALLY SEEN ON THE ORDER OF 40-50K, BUT THE NUMBER WILL INCREASE OVER TIME
# NOTE THAT SOMETIMES YOU HAVE TO RESTART INSTANCE TO RUN THIS PART
# BECAUSE NB SEEMS TO BE STUCK HANGING, BUT PDFS HAVE ACTUALLY BEEN UPLOADED
!gsutil du {BUCKET} | wc -l